In [ ]:
import os
import SimpleITK as sitk
for ct_file in os.listdir('/home/yanxin/sorted_data_16/test/input_stride_1_all/'):
    
    ct = sitk.ReadImage('/home/yanxin/sorted_data_16/test/input_stride_1_all/'+ct_file, sitk.sitkInt16)
    ct_array = sitk.GetArrayFromImage(ct)
    print(ct_array.min(),ct_array.max())
#     print(ct_array.shape)

# #     seg_file = ct_file.replace('volume', 'segmentation')
#     seg = sitk.ReadImage('/home/yanxin/sorted_data_16/train/gt/', sitk.sitkInt16)
#     seg_array = sitk.GetArrayFromImage(seg)

In [ ]:
import os
import SimpleITK as sitk
# for ct_file in os.listdir('/home/yanxin/sorted_data_16/train/gt/'):
    
ct = sitk.ReadImage('/home/yanxin/data_ct/0Venous_tra_5mm_roi.nii', sitk.sitkInt16)
ct_array = sitk.GetArrayFromImage(ct)
print(ct_array.shape)

In [ ]:
ct_array_list=[]

In [ ]:
ct_array_list.append(ct_array[0:16, :, :])
ct_array_list.append(ct_array[16:32, :, :])
ct_array_list.append(ct_array[-16:, :, :])

In [ ]:
import torch

In [ ]:
for ct_array in ct_array_list:
    ct_tensor = torch.FloatTensor(ct_array)
    print(ct_tensor.shape)
    ct_tensor = ct_tensor.unsqueeze(dim=0)
    print(ct_tensor.shape)
    ct_tensor = ct_tensor.unsqueeze(dim=0)
    print(ct_tensor.shape)
    ct_tensor = ct_tensor.squeeze()
    print(ct_tensor.shape)

In [ ]:
ct_array=np.zeros([16,256,256])

In [ ]:
alist=[]
alist.append(ct_array[2,:,:])
alist.append(ct_array[3,:,:])

In [ ]:
pred_seg = np.concatenate(alist, axis=0)

In [ ]:
pred_seg.shape

In [ ]:
aa=np.array(alist)

In [ ]:
pred_seg = (np.array(alist)> 0).astype(np.int16)

In [ ]:
pred_seg = np.concatenate(alist, axis=0)

In [ ]:
import os
import SimpleITK as sitk
import matplotlib.pyplot as plt

In [ ]:
for ct_slice in range(ct_array.shape[0]):
    pred_slice=[]
    for s in start_end_slice_list:
        if ct_slice>=s[0] and ct_slice<=s[1]:
            pred_slice.append(outputs_list[start_end_slice_list.index(s)][ct_slice-s[0],:,:])
    pred_seg = np.concatenate(pred_slice, axis=0)

In [ ]:
import os
import SimpleITK as sitk
import matplotlib.pyplot as plt
for i in range(120, 150):
    ct = sitk.ReadImage('/home/yanxin/data_ct/' + str(i) + 'Venous_tra_5mm.nii')
    ct_array = sitk.GetArrayFromImage(ct)
    plt.show[ct_array[0]]

In [ ]:
def plot_v(index):
    ct = sitk.ReadImage('/home/yanxin/data_ct/' + str(index) + 'Venous_tra_5mm.nii')
    ct_array = sitk.GetArrayFromImage(ct)
    ct_gt = sitk.ReadImage('/home/yanxin/data_ct/' + str(index) + 'Venous_tra_5mm_roi.nii')
    ct_array_gt = sitk.GetArrayFromImage(ct_gt)
    ct_pred = sitk.ReadImage('/home/yanxin/DenseV_net/pred/'+ str(index) + 'Venous_tra_5mm_pred.nii')
    ct_array_pred = sitk.GetArrayFromImage(ct_pred)
    # plt.figure(figsize=(8, 8))
    print(ct_array.shape)
    # for i in range(ct_array.shape[0]):
    for i in range(38):
        if ct_array_pred[i].max()==1 or ct_array_gt[i].max()==1:
            plt.figure(figsize=(10, 10))
#             plt.subplot(1,2,1)
#             plt.imshow(ct_array[i],cmap='gray')
#             plt.subplot(1,2,2)
        #     plt.imshow(ct_array_gt[i],cmap='gray')
        #     plt.subplot(1,4,3)
        #     plt.imshow(ct_array_pred[i],cmap='gray')
        #     plt.subplot(1,4,4)
            plt.imshow(marked_roi(ct_array[i],ct_array_pred[i],ct_array_gt[i]))
            plt.show()


In [ ]:
plot_v(148)

In [ ]:
from skimage import io, measure, draw,color
import skimage
# from parallel import parallel_process, agg_results
from multiprocessing import Manager
# from config import Config
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
def marked_roi(image, mask_pred,mask_gt):
    _min = image.min()
    _max = image.max()
    image = (image - _min) / (_max - _min)
    image = skimage.color.gray2rgb(image)
    contour_gt = measure.find_contours(mask_gt, 0)
    contour_pred = measure.find_contours(mask_pred, 0)
#     print(len(contour_gt))
#     print(len(contour_pred))
    if len(contour_gt)==0 and len(contour_pred)==0:
        return image
    if len(contour_gt) >= 1:
        for gt_lk in contour_gt:
            contour_pixel_gt = (np.array([int(c[0]) for c in gt_lk]), np.array([int(c[1]) for c in gt_lk]))
#             marked = image.copy()
            image[contour_pixel_gt] = [1, 0, 0] # 目标是红
    if len(contour_pred) >= 1:
        for pred_lk in contour_pred:
            contour_pixel_pred = (np.array([int(c[0]) for c in pred_lk]), np.array([int(c[1]) for c in pred_lk]))
#             marked = marked.copy()
            image[contour_pixel_pred] = [0, 0, 1]
#         contour_pixel_pred = (np.array([int(c[0]) for c in contour_pred[0]]), np.array([int(c[1]) for c in contour_pred[0]]))
#     print(len(contour_pixel))
#         marked[contour_pixel_pred] = [1, 0, 0]
    return image